In [27]:
from time import time
import numpy as np
import pandas as pd

import re
import nltk
from nltk.corpus import stopwords
from gensim.models import word2vec

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [36]:
df = pd.read_csv("./labeled_rutoxic.csv", delimiter=',', header=0, names=['sentence', 'label'])

print('В наборе предложений: \n',df.shape[0])

В наборе предложений: 
 14412


In [37]:
print('toxic:', df[df['label'] > 0]['label'].count())
print('not toxic:', df[df['label'] < 1]['label'].count())

toxic: 4826
not toxic: 9586


## Разбиение на тестовые и обучающие

In [38]:
X = df.iloc[:,0]#
y = df.iloc[:,1]#

train , test , y_train, y_test = train_test_split(X, y, test_size=0.3) # и отдаем 30% на тест, остальное на обучен

## Разбиение на слова с помощью модели Word2Vec

In [39]:
import re
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Инициализация лемматизатора
lemmatizer = WordNetLemmatizer()

# Метод преобразования текста в слова
def text_to_words(raw_text, remove_stopwords=False):
    # Удаление лишних символов, оставляем только буквы и цифры
    letters_and_numbers_only = re.sub("[^0-9а-яА-Я]", " ", raw_text)

    # Приведение к нижнему регистру и токенизация
    words = letters_and_numbers_only.lower().split()

    # Удаление стоп-слов, если требуется
    if remove_stopwords:
        stops = set(stopwords.words("russian"))
        words = [w for w in words if not w in stops]

    # Лемматизация
    words = [lemmatizer.lemmatize(word) for word in words]

    return words

sentences_train = train.apply(text_to_words, remove_stopwords=False)
sentences_test = test.apply(text_to_words, remove_stopwords=False)
print("\nSentences_train:")
print(sentences_train[:10])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\IVAN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\IVAN\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!



Sentences_train:
1590     [сдается, мне, в, этой, конторе, нанимают, тех...
243      [я, дурачок, купил, игра, гавно, однообразно, ...
12632    [ну, спасибо, столько, раз, на, это, и, вживую...
239      [ты, троллишь, если, ты, такой, правильный, пл...
10500    [мой, личный, топ, 3, трилогия, ме, дилогия, к...
4184     [ну, например, там, было, написано, что, я, не...
7928     [сомневаюсь, что, их, когда, нибудь, все, откр...
2549            [интересно, почему, сейчас, не, обсуждают]
13705    [дрожин, кусок, говна, с, мерзким, ебалом, ско...
12803    [вот, расположение, только, я, не, местный, та...
Name: sentence, dtype: object


In [40]:

num_features = 300
min_word_count = 40
num_workers = 4
context = 20
downsampling = 1e-3
model = word2vec.Word2Vec(sentences_train, workers=num_workers, vector_size=num_features, min_count = min_word_count, window = context, sample = downsampling)
print(sentences_train[:1])
print(model.wv[1].shape)
print(model.wv[1])

1590    [сдается, мне, в, этой, конторе, нанимают, тех...
Name: sentence, dtype: object
(300,)
[-9.73262712e-02  6.57440200e-02 -1.32309929e-01  3.99766788e-02
  1.35120109e-01 -2.58908629e-01 -1.65641308e-03  2.61599869e-01
  1.77603990e-01 -1.92030847e-01  4.38472509e-01  8.04604366e-02
 -4.32155095e-02 -1.59699544e-01 -4.70517464e-02 -1.63453922e-01
  2.01564759e-01  8.51944238e-02  2.57914841e-01  2.31918290e-01
 -1.55077921e-02  5.73461577e-02  3.31430107e-01  3.72655660e-01
  2.81931937e-01  2.12377906e-01 -2.33935580e-01 -5.03270030e-02
  4.14911881e-02 -3.09999198e-01 -2.17044838e-02 -3.03304672e-01
 -7.79279135e-03 -6.05441444e-02 -5.33397794e-02  2.67152160e-01
 -2.60980725e-01 -2.34153882e-01 -3.16376239e-01 -2.37054005e-02
 -1.06440969e-01  3.14049184e-01  9.73007828e-02  5.15800789e-02
  1.69119850e-01 -6.27417937e-02  5.10144942e-02  1.41896471e-01
  5.86675946e-03 -4.02600057e-02  5.54032959e-02  4.99141105e-02
 -3.14812958e-02 -2.85249591e-01 -3.01640313e-02  2.03316081

In [41]:
# получение векторного представления
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0

    index2word_set = set(model.wv.index_to_key)

    for word in words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec, model.wv[word])

    if nwords == 0:
        nwords = 1
    featureVec = np.divide(featureVec, nwords)
    return featureVec

# получение среднего векторного простнраства для предложения
def getAvgFeatureVecs(reviews, model, num_features):
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype="float32")
    counter = 0
    for review in reviews:
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter = counter + 1
    return reviewFeatureVecs

f_matrix_train = getAvgFeatureVecs(sentences_train, model, num_features)
f_matrix_test = getAvgFeatureVecs(sentences_test, model, num_features)
print("Числовые вектора предложений полученные из усреднения его векторов слов:")
print(f_matrix_train[:10])

Числовые вектора предложений полученные из усреднения его векторов слов:
[[ 0.01122577  0.1648979  -0.02985618 ... -0.0644412   0.05634512
  -0.10626563]
 [ 0.01660705  0.15107937 -0.02373466 ... -0.06420577  0.06979375
  -0.10469719]
 [ 0.03489444  0.17293017 -0.0200489  ... -0.05951793  0.07664739
  -0.1255422 ]
 ...
 [ 0.04794977  0.1994577   0.00728247 ... -0.0754095   0.18881232
  -0.09791721]
 [ 0.03975372  0.16338009 -0.02574402 ... -0.05212484  0.04833128
  -0.13754559]
 [ 0.0510293   0.20172569  0.01441811 ... -0.08039115  0.21878174
  -0.08956188]]


## Обучение  MLPClassifier

In [42]:
model = []
#adam солвер это стохастически градиентный оптимизатор
m = MLPClassifier(solver='adam', hidden_layer_sizes=(100,50,20), random_state=1)
model.append(m)

print(model)

[MLPClassifier(hidden_layer_sizes=(100, 50, 20), random_state=1)]


In [43]:
batch_size = 200
total_rows = f_matrix_train.shape[0]
duration = 0
start_train = time()
pos = 0
classes = [0.0, 1.0]
while duration < 10 and pos < total_rows:
    if pos+batch_size > total_rows:
        batch_size = total_rows-pos
    X_p = f_matrix_train[pos:pos+batch_size]
    y_p = y_train.values[pos:pos+batch_size]
    model[0].partial_fit(X_p, y_p, classes)
    pos = pos + batch_size
    duration = time() - start_train
    if pos == total_rows:
        pos = 0
        batch_size = 10000
print('done')

done


In [44]:
## Сохранение результатов и расчет ошибки

In [45]:
y_test_values=y_test.values
predicted_results = model[0].predict_proba(f_matrix_test)
predicted_results = np.where(predicted_results[:,0]>predicted_results[:,1], 0.0,1.0)

sum_errors =sum(y_test_values - predicted_results)
accuracy = (len(y_test_values) - sum_errors) / len(y_test_values) *100
print('count test values', len(y_test_values))
print('sum_errors', sum_errors)
print('accuracy', accuracy)

count test values 4324
sum_errors 814.0
accuracy 81.17483811285847


In [46]:
saved_result = pd.DataFrame({'text':test.values,
 'expected':  y_test_values,
 'predicted': predicted_results})

In [47]:
saved_result.to_csv('result.csv', encoding='utf-8', index=False)

PermissionError: [Errno 13] Permission denied: 'result.csv'